In [1]:
# run this cell to import nltk
import nltk
from os import getcwd
import numpy as np
import pandas as pd
import re
from collections import Counter

In [2]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer

def removePatterns(tweet):    
    patterns = ['URL', '@USER', '\'ve', 'n\'t', '\'s', '\'m']
    for pattern in patterns:
        tweet = tweet.replace(pattern, '')
    return re.sub(r'[^a-zA-Z]', ' ', tweet)

def removeEmoji(tweet):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',tweet)
    
def tweetTokenize(tweet):
    return word_tokenize(tweet.lower())

def remove_stop_words(tokens):
    stop_words = set(stopwords.words('english'))
    processed_tokens = []
    for token in tokens:  
        if token not in stop_words:  
            processed_tokens.append(token)  
    return processed_tokens

def stem_and_lem(tokens):
    stemmer = PorterStemmer()
    wordnet_lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for token in tokens:
        token = wordnet_lemmatizer.lemmatize(token)
        #token = stemmer.stem(token)
        if len(token) > 1:
            clean_tokens.append(token)
    return clean_tokens


In [3]:
import os
dirFiles = os.listdir('E:\\college\\level.4\\first term\\IR\\project\\myPartProject\\DataSets')
print(dirFiles)


['0.txt', '1.txt', '2.txt', '3.txt', '4.txt', '5.txt', '6.txt', '7.txt', '8.txt', '9.txt']


In [4]:
list_of_tweets = []
list_of_processed_tweets = []

for file in dirFiles:
    a_file = open("E:\\college\\level.4\\first term\\IR\\project\\myPartProject\\DataSets\\"+ file, "r")
    for line in a_file:
        list_of_tweets.append(line)
        tweet = removePatterns(line)
        tweet = removeEmoji(tweet)
        tweet = tweetTokenize(tweet)
        tokens = remove_stop_words(tweet)
        tokens = stem_and_lem(tokens)
        ##line_list = process_tweet(line)
        list_of_processed_tweets.append(tokens)
    a_file.close()
    
print(list_of_processed_tweets[0])

['nlp', 'based', 'information', 'retrieval', 'system']


In [5]:
# test the function below
print('This is an example of a tweet: \n', list_of_tweets[4])
print('\nThis is an example of the processed version of the tweet: \n', list_of_processed_tweets[4])

This is an example of a tweet: 
 In other words, a textual information retrieval system carries out the following tasks in response to a user's query.

This is an example of the processed version of the tweet: 
 ['word', 'textual', 'information', 'retrieval', 'system', 'carry', 'following', 'task', 'response', 'user', 'query']


# Positional Index

In [6]:
# Initialize the file no. 
fileno = 0
  
# Initialize the dictionary. 
pos_index = {} 
  
# Initialize the file mapping (fileno -> file name). 
file_map = {} 

for tweet in list_of_processed_tweets:
    for pos, term in enumerate(tweet):
        # If term already exists in the positional index dictionary. 
        if term in pos_index: 
            # Increment total freq by 1. 
            pos_index[term][0] = pos_index[term][0] + 1
            # Check if the term has existed in that DocID before. 
            if fileno in pos_index[term][1]: 
                pos_index[term][1][fileno].append(pos) 
            else: 
                pos_index[term][1][fileno] = [pos] 
        # If term does not exist in the positional index dictionary  
        # (first encounter). 
        else: 
            # Initialize the list. 
            pos_index[term] = [] 
            # The total frequency is 1. 
            pos_index[term].append(1) 
            # The postings list is initially empty. 
            pos_index[term].append({})       
            # Add doc ID to postings list. 
            pos_index[term][1][fileno] = [pos] 
    # Map the file no. to the file name. 
    file_map[fileno] = tweet

    # Increment the file no. counter for document ID mapping               
    fileno += 1

# Sample positional index to test the code. 
sample_pos_idx = pos_index["system"] 
print("Positional Index") 
print(sample_pos_idx) 
  
file_list = sample_pos_idx[1] 
print("Filename, [Positions]") 
for fileno, positions in file_list.items(): 
    print(dirFiles[fileno],"   ", positions) 

Positional Index
[5, {0: [4], 4: [4], 6: [3, 9], 9: [2]}]
Filename, [Positions]
0.txt     [4]
4.txt     [4]
6.txt     [3, 9]
9.txt     [2]


# Vector Space Model

In [96]:
query = "NLP and Information Retrieval System"
list_word_in_quey = []
word = removePatterns(query)
word = tweetTokenize(word)
word = remove_stop_words(word)
query_list = stem_and_lem(word)
query_list


['nlp', 'information', 'retrieval', 'system']

In [75]:
doc0 = list_of_processed_tweets[0]
doc1 = list_of_processed_tweets[1]
doc2 = list_of_processed_tweets[2]
doc3 = list_of_processed_tweets[3]
doc4 = list_of_processed_tweets[4]
doc5 = list_of_processed_tweets[5]
doc6 = list_of_processed_tweets[6]
doc7 = list_of_processed_tweets[7]
doc8 = list_of_processed_tweets[8]
doc9 = list_of_processed_tweets[9]

In [76]:
doc9

['love', 'information', 'system']

In [97]:
wordSet = set(doc0).union(set(doc1)).union(set(doc2)).union(set(doc3)).union(set(doc4)).union(set(doc5)).union(set(doc6)).union(set(doc7)).union(set(doc8)).union(set(doc9)).union(set(query_list))

In [98]:
wordDict0 = dict.fromkeys(wordSet, 0) 
wordDict1 = dict.fromkeys(wordSet, 0)
wordDict2 = dict.fromkeys(wordSet, 0) 
wordDict3 = dict.fromkeys(wordSet, 0)
wordDict4 = dict.fromkeys(wordSet, 0) 
wordDict5 = dict.fromkeys(wordSet, 0)
wordDict6 = dict.fromkeys(wordSet, 0) 
wordDict7 = dict.fromkeys(wordSet, 0)
wordDict8 = dict.fromkeys(wordSet, 0) 
wordDict9 = dict.fromkeys(wordSet, 0)
queryDict = dict.fromkeys(wordSet, 0)

In [99]:
for word in doc0:
    wordDict0[word]+=1
    
for word in doc1:
    wordDict1[word]+=1

for word in doc2:
    wordDict2[word]+=1
    
for word in doc3:
    wordDict3[word]+=1
    
for word in doc4:
    wordDict4[word]+=1

for word in doc5:
    wordDict5[word]+=1
    
for word in doc6:
    wordDict6[word]+=1

for word in doc7:
    wordDict7[word]+=1
    
for word in doc8:
    wordDict8[word]+=1

for word in doc9:
    wordDict9[word]+=1

for word in query_list:
    queryDict[word]+=1

In [100]:
import pandas as pd
pd.DataFrame([wordDict0, wordDict1, wordDict2, wordDict3, wordDict4, wordDict5, wordDict6, wordDict7, wordDict8, wordDict9, queryDict])

,software,without,deal,human,repetition,large,document,use,location,consists,...,request,may,collection,premeditation,address,content,response,information,variety,processing
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,0,0,0,0,0,1,2,0,0,0,...,1,0,1,0,1,1,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
5,1,0,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,3,0,0
6,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,1,0
7,0,1,0,1,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


TF: term frequency

In [101]:
def computeTF(wordDict, bow):
    tfDict = {}
    bowCount = len(bow)
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

In [102]:
tfBow0 = computeTF(wordDict0, doc0)
tfBow1 = computeTF(wordDict1, doc1)
tfBow2 = computeTF(wordDict2, doc2)
tfBow3 = computeTF(wordDict3, doc3)
tfBow4 = computeTF(wordDict4, doc4)
tfBow5 = computeTF(wordDict5, doc5)
tfBow6 = computeTF(wordDict6, doc6)
tfBow7 = computeTF(wordDict7, doc7)
tfBow8 = computeTF(wordDict8, doc8)
tfBow9 = computeTF(wordDict9, doc9)
tfQuery = computeTF(queryDict, query_list)

In [103]:
pd.DataFrame([tfBow0, tfBow1, tfBow2, tfBow3, tfBow4, tfBow5, tfBow6, tfBow7, tfBow8, tfBow9, tfQuery])

,software,without,deal,human,repetition,large,document,use,location,consists,...,request,may,collection,premeditation,address,content,response,information,variety,processing
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.2
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.133333,0.000000,0.000000,0.000000,...,0.066667,0.000000,0.066667,0.000000,0.066667,0.066667,0.000000,0.066667,0.000000,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.090909,0.000000,0.0
5,0.055556,0.000000,0.055556,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,0.000000,...,0.000000,0.055556,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.0
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.058824,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.0
7,0.000000,0.071429,0.000000,0.071429,0.071429,0.000000,0.000000,0.071429,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.0


IDF: Inverse Document Frequency

In [104]:
import math
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
        
    return idfDict

In [105]:
idfs = computeIDF([wordDict0, wordDict1, wordDict2, wordDict3, wordDict4, wordDict5, wordDict6, wordDict7, wordDict8, wordDict9, queryDict])

tf-idf:

In [106]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf

In [107]:
tfidfBow0 = computeTF(tfBow0, idfs)
tfidfBow1 = computeTF(tfBow1, idfs)
tfidfBow2 = computeTF(tfBow2, idfs)
tfidfBow3 = computeTF(tfBow3, idfs)
tfidfBow4 = computeTF(tfBow4, idfs)
tfidfBow5 = computeTF(tfBow5, idfs)
tfidfBow6 = computeTF(tfBow6, idfs)
tfidfBow7 = computeTF(tfBow7, idfs)
tfidfBow8 = computeTF(tfBow8, idfs)
tfidfBow9 = computeTF(tfBow9, idfs)
tfidfQuery = computeTF(tfQuery, idfs)

In [108]:
pd.DataFrame([tfidfBow0, tfidfBow1, tfidfBow2, tfidfBow3, tfidfBow4, tfidfBow5, tfidfBow6, tfidfBow7, tfidfBow8, tfidfBow9, tfidfQuery])

,software,without,deal,human,repetition,large,document,use,location,consists,...,request,may,collection,premeditation,address,content,response,information,variety,processing
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003226,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002304,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003226,0.000000,0.003226
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.001075,0.002151,0.000000,0.000000,0.000000,...,0.001075,0.000000,0.001075,0.000000,0.001075,0.001075,0.000000,0.001075,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001466,0.001466,0.000000,0.000000
5,0.000896,0.000000,0.000896,0.000000,0.000000,0.000000,0.000896,0.000000,0.000000,0.000000,...,0.000000,0.000896,0.000000,0.000000,0.000000,0.000000,0.000000,0.002688,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000949,0.000949,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000949,0.000000
7,0.000000,0.001152,0.000000,0.001152,0.001152,0.000000,0.000000,0.001152,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.001152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005376,0.000000,0.000000


Semilarity between Query And Documents 

In [109]:
print(type(tfidfQuery))

<class 'dict'>


In [110]:
q1 = []
for key, value in tfidfQuery.items():
    q1.append(value)

In [111]:
f0 = []
for key, value in tfidfBow0.items():
    f0.append(value)

f1 = []
for key, value in tfidfBow1.items():
    f1.append(value)
    
f2 = []
for key, value in tfidfBow2.items():
    f2.append(value)
    
f3 = []
for key, value in tfidfBow3.items():
    f3.append(value)

f4 = []
for key, value in tfidfBow4.items():
    f4.append(value)
    
f5 = []
for key, value in tfidfBow5.items():
    f5.append(value)
    
f6 = []
for key, value in tfidfBow6.items():
    f6.append(value)

f7 = []
for key, value in tfidfBow7.items():
    f7.append(value)
    
f8 = []
for key, value in tfidfBow8.items():
    f8.append(value)
    
f9 = []
for key, value in tfidfBow9.items():
    f9.append(value)

In [112]:
allDocs = f0 , f1 , f2, f3, f4, f5, f6, f7, f8, f9

In [113]:
from numpy import dot
from numpy.linalg import norm
cosine = lambda v1, v2: dot(v1, v2) / (norm(v1) * norm(v2))
cosine([1, 1, 1, 0], [1, 1, 0, 1])

0.6666666666666667

In [114]:
cosine(q1, f0)

0.8944271909999157

In [115]:
i=0
for f in allDocs:
    x = cosine(q1, f)
    print ("similarity between query and file ", i, "is: ", x)
    i+=1

similarity between query and file  0 is:  0.8944271909999157
similarity between query and file  1 is:  0.6666666666666665
similarity between query and file  2 is:  0.44721359549995787
similarity between query and file  3 is:  0.24253562503633294
similarity between query and file  4 is:  0.45226701686664544
similarity between query and file  5 is:  0.4902903378454601
similarity between query and file  6 is:  0.32732683535398854
similarity between query and file  7 is:  0.0
similarity between query and file  8 is:  0.3535533905932738
similarity between query and file  9 is:  0.5773502691896257
